In [5]:
import torch
import jovian
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

ModuleNotFoundError: No module named 'jovian'

In [6]:
!pip install jovian

  Created wheel for uuid: filename=uuid-1.30-py3-none-any.whl size=6501 sha256=2370c294205a81b1cf43f98407054f8f68bbb7bea6c20d1e00709e9ecca73262
  Stored in directory: c:\users\nani\appdata\local\pip\cache\wheels\2a\ea\87\dd57f1ecb4f0752f3e1dbf958ebf8b36d920d190425bcdc24d
Successfully built uuid


In [7]:
import jovian
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

<IPython.core.display.Javascript object>

In [8]:
DATA_FILENAME = "car_data.csv"
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [9]:
your_name = "Aman Kharwal" # at least 5 characters
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.Year = dataframe.Year * ord(rand_str[1])/100.
    # scale target
    dataframe.Selling_Price = dataframe.Selling_Price * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['Car_Name'], axis=1)
    return dataframe

dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
2,ciaz,2198.53,7.0325,9.85,6900,Petrol,Dealer,Manual,0
128,Honda CB Hornet 160R,2198.53,0.7760,0.87,3000,Petrol,Individual,Manual,0
286,jazz,2197.44,5.4805,7.90,28569,Petrol,Dealer,Manual,0
272,city,2196.35,7.2750,10.00,27600,Petrol,Dealer,Manual,0
205,grand i10,2197.44,5.0925,5.70,3493,Petrol,Dealer,Manual,1


In [11]:
input_cols = ["Year","Present_Price","Kms_Driven","Owner"]
categorical_cols = ["Fuel_Type","Seller_Type","Transmission"]
output_cols = ["Selling_Price"]

In [12]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[2.19853e+03, 9.85000e+00, 6.90000e+03, 0.00000e+00],
        [2.19853e+03, 8.70000e-01, 3.00000e+03, 0.00000e+00],
        [2.19744e+03, 7.90000e+00, 2.85690e+04, 0.00000e+00],
        ...,
        [2.19853e+03, 3.62300e+01, 6.00000e+03, 0.00000e+00],
        [2.18872e+03, 5.20000e-01, 5.00000e+05, 0.00000e+00],
        [2.19635e+03, 8.40000e-01, 5.80000e+04, 0.00000e+00]]),
 array([[ 7.0325],
        [ 0.776 ],
        [ 5.4805],
        [ 7.275 ],
        [ 5.0925],
        [ 0.4656],
        [ 2.7645],
        [ 0.3395],
        [ 1.6975],
        [ 4.8015],
        [ 8.9725],
        [ 5.141 ],
        [ 3.783 ],
        [ 8.827 ],
        [ 4.268 ],
        [ 3.3853],
        [19.1575],
        [ 7.0325],
        [10.9125],
        [12.513 ],
        [14.2881],
        [ 4.753 ],
        [ 5.0925],
        [ 3.1525],
        [ 7.2265],
        [ 0.5335],
        [ 2.813 ],
        [ 0.3007],
        [ 0.2619],
        [ 7.178 ],
        [ 0.582 ],
        [ 5.9655],
     

In [13]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

dataset = TensorDataset(inputs, targets)
train_ds, val_ds = random_split(dataset, [228, 57])
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [14]:
input_size = len(input_cols)
output_size = len(output_cols)

class CarsModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                         # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))
            
model = CarsModel()

list(model.parameters())

[Parameter containing:
 tensor([[-0.0546, -0.3798,  0.0931,  0.2728]], requires_grad=True),
 Parameter containing:
 tensor([0.3128], requires_grad=True)]

In [15]:
# Eval algorithm
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

# Fitting algorithm
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

# Check the initial value that val_loss have
result = evaluate(model, val_loader)
print(result)

{'val_loss': 3372.395751953125}


In [16]:
# Start with the Fitting
epochs = 90
lr = 1e-8
history1 = fit(epochs, lr, model, train_loader, val_loader)

c:\users\nani\appdata\local\programs\python\python37\lib\site-packages\torch\autograd\__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


Epoch [20], val_loss: 2809.7595
Epoch [40], val_loss: 2247.7168
Epoch [60], val_loss: 1686.2509
Epoch [80], val_loss: 1126.1378
Epoch [90], val_loss: 849.4143


In [17]:
# Train repeatdly until have a 'good' val_loss
epochs = 20
lr = 1e-9
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 794.5378


In [18]:
# Prediction Algorithm
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

# Testing the model with some samples
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([2.1974e+03, 5.4000e-01, 1.4000e+04, 0.0000e+00])
Target: tensor([0.4365])
Prediction: tensor([214.0282])


In [19]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([2.1953e+03, 6.8000e+00, 1.6500e+04, 0.0000e+00])
Target: tensor([5.1410])
Prediction: tensor([272.8781])
